# Highway Vehicle Counting Practice Exercise


In this exercise, you will use the YOLO (You Only Look Once) object detection model to analyze a video of highway traffic.
Your task is to count the number of cars that are leaving the highway (coming toward the camera) on the right side and the number of cars that are joining the road on the left side. The video can be found under `Datasets/Example.mp4`.


In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.2/872.2 kB 11.3 MB/s eta 0:00:00


## Load the YOLO Model

In [2]:
import cv2
from ultralytics import YOLO, solutions
from google.colab.patches import cv2_imshow

model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 25.1MB/s]


## Prepare the Video Capture

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Tuwaiq/Week 6 /Practice Exercise/Datasets/Example.mp4')
assert cap.isOpened(), 'Error reading video file'
width, height, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))


## Prepare Video Writer to Store the Output

In [5]:
# output path
output_path = '/content/output_video_with_predictions_t.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Process Video Frames and Identify Vehicles on the Right and Left Sides

For each frame in the video, use the YOLO model to detect and track vehicles. You'll need to write a loop that processes each frame and applies the YOLO model.
In each frame, after detecting the vehicles, determine whether they are on the left or right side of the highway.
You can use the position of the bounding boxes provided by YOLO to do this.
* The video should display bounding boxes around the detected objects.
* The video should display the confidence along side with the object id and class id of each detected and tracked object.
* The video display the number of vehicles on the left side.
* The video display the number of vehicles on the right side.
* The video should display the line in which you counted the objects that have crossed it and counted.

In [8]:
color = (30, 185, 223)
thickness = 1
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_thickness = 1

# Define mid points
mid_x = width // 2
mid_y = height // 2

left_counter = 0
right_counter = 0

ret = True
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    class_count = {}
    results = model.track(frame, persist=True)

    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0
            id = obj.id[0].cpu().numpy() if obj.id is not None else -1

            if class_id not in class_count:
                class_count[class_id] = 0
            class_count[class_id] += 1

            x1, y1, x2, y2 = map(int, bbox)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            label = f'ID: {id}, Class: {class_id}, Conf: {conf:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)

            # Count vehicles crossing the line
            if x2 < mid_x and y1 < mid_y and y2 > mid_y:  # Left to right
                left_counter += 1
            elif x1 > mid_x and y1 < mid_y and y2 > mid_y:  # Right to left
                right_counter += 1

    # Draw the vertical line
    cv2.line(frame, (mid_x, 0), (mid_x, height), color, thickness)

    # Display counts on the frame
    cv2.putText(frame, f'Cars Leaving: {right_counter}', (10, height - 40), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'Cars Joining: {left_counter}', (10, height - 20), font, font_scale, color, font_thickness, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()


0: 384x640 5 cars, 51.6ms
Speed: 1.7ms preprocess, 51.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 55.2ms
Speed: 2.5ms preprocess, 55.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 57.7ms
Speed: 2.4ms preprocess, 57.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 55.4ms
Speed: 2.0ms preprocess, 55.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 54.1ms
Speed: 1.9ms preprocess, 54.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 57.4ms
Speed: 2.6ms preprocess, 57.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 59.9ms
Speed: 2.5ms preprocess, 59.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 54.9ms
Speed: 1.7ms preprocess, 54.9ms inference, 0.8ms postprocess per image at shap